In [ ]:
import time
import numpy as np
import pandas as pd
# from sklearn import metrics

from pycaret.datasets import get_data
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [ ]:
y = get_data('airline', verbose=False)

In [ ]:
fh = 12 # or alternately fh = np.arange(1,13)
fold = 3

# Available Models

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=fh)
exp.models()

# Plotting Data

In [ ]:
# Without an estimator argument, this will plot the original dataset
exp.plot_model(plot="ts")

In [ ]:
# ACF and PACF for the original dataset
exp.plot_model(plot="acf")
exp.plot_model(plot="pacf")

In [ ]:
# Show the train-test splits on the dataset
# Internally split - len(fh) as test set, remaining used as test set
exp.plot_model(plot="splits-tt")

# Show the Cross Validation splits inside the train set
exp.plot_model(plot="splits_cv")

# Tests

In [ ]:
exp.check_stats()

In [ ]:
# Options are: 'all', 'stat_summary', 'white_noise' 'stationarity', 'adf', 'kpss', 'normality'
# Setting denotes alpha value used (for most tests). For white noise, it denotes the lags used to test
exp.check_stats(test='stationarity')

In [ ]:
# For white noise, Setting it denotes the lags used to test
exp.check_stats(test='white_noise')

In [ ]:
# You can change alpha if needed (would not recommend though)
exp.check_stats(test='stationarity', alpha = 0.2)

# Flow example

## Common Setup

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=fh, fold=fold, session_id=42)

In [ ]:
y_train = exp.get_config("y_train")
y_test = exp.get_config("y_test")

## Manual Create

### Classical Statistical Models

In [ ]:
model = exp.create_model("exp_smooth")
y_predict = exp.predict_model(model)
exp.plot_model(estimator=model, plot='predictions')
round(mean_absolute_percentage_error(y_predict, y_test), 4)

In [ ]:
# Fixed Grid Search
tuned_model = exp.tune_model(model)
print(model)
print(tuned_model)

In [ ]:
# Random Grid Search
tuned_model = exp.tune_model(model, search_algorithm="random")
print(model)
print(tuned_model)

In [ ]:
y_predict = exp.predict_model(tuned_model)
exp.plot_model(estimator=tuned_model, plot='predictions')
round(mean_absolute_percentage_error(y_predict, y_test), 4)

In [ ]:
# Random Grid Search with different number of iterations
tuned_model = exp.tune_model(model, search_algorithm="random", n_iter=5)
print(model)
print(tuned_model)

### Reduced Regressors: Random Forest (with internal conditional deseasonalize and detrending)

In [ ]:
model = exp.create_model("rf_cds_dt")
y_predict = exp.predict_model(model)
exp.plot_model(estimator=model, plot='predictions')
round(mean_absolute_percentage_error(y_predict, y_test), 4)

In [ ]:
# Fixed Grid Search
tuned_model = exp.tune_model(model)
print(model)
print(tuned_model)

In [ ]:
# Random Grid Search
tuned_model = exp.tune_model(model, search_algorithm="random")
print(model)
print(tuned_model)

In [ ]:
y_predict = exp.predict_model(tuned_model)
exp.plot_model(estimator=tuned_model, plot='predictions')
round(mean_absolute_percentage_error(y_predict, y_test), 4)

## Auto Create

### Compare Models

In [ ]:
best_baseline_models = exp.compare_models(fold=fold, sort='smape', n_select=3)
best_baseline_models

In [ ]:
compare_metrics = exp.pull()
compare_metrics

* Note that some models like BATS and TBATS are disabled by default. 
* You can enable them by setting `turbo = False`

In [ ]:
# _ = exp.compare_models(fold=fold, sort='smape', n_select=3, turbo=False)

### Tune Best Models

In [ ]:
best_tuned_models = [exp.tune_model(model) for model in best_baseline_models]
best_tuned_models

### Blend Best Models

#### Mean Blender

In [ ]:
mean_blender = exp.blend_models(best_tuned_models, method='mean')

In [ ]:
y_predict = exp.predict_model(mean_blender)
# exp.plot_model(estimator=mean_blender, plot='predictions')
plot_series(y, y_train, y_predict[0], labels=['All', 'Train', 'Predictions'])
round(mean_absolute_percentage_error(y_predict, y_test), 4)

#### Median Blender

In [ ]:
median_blender = exp.blend_models(best_tuned_models, method='median')

In [ ]:
y_predict = exp.predict_model(median_blender)
exp.plot_model(estimator=median_blender, plot='predictions')
round(mean_absolute_percentage_error(y_predict, y_test), 4)

#### Voting Blender

In [ ]:
top_model_metrics = compare_metrics.iloc[0:3]['SMAPE']
display(top_model_metrics)

top_model_weights = 1 - top_model_metrics/top_model_metrics.sum()
display(top_model_weights)

In [ ]:
voting_blender = exp.blend_models(best_tuned_models, method='voting', weights=top_model_weights.values)

In [ ]:
y_predict = exp.predict_model(voting_blender)
# exp.plot_model(estimator=voting_blender, plot='predictions')
plot_series(y, y_train, y_predict[0], labels=['All', 'Train', 'Predictions'])
round(mean_absolute_percentage_error(y_predict, y_test), 4)

## Prediction Customization

In [ ]:
model = exp.create_model("auto_arima")

In [ ]:
# Default prediction
exp.predict_model(model)

In [ ]:
# With Prediction Interval (default alpha = 0.05)
exp.predict_model(model, return_pred_int=True)

In [ ]:
# With Prediction Interval (custom alpha = 0.2)
exp.predict_model(model, return_pred_int=True, alpha=0.2)

In [ ]:
# Increased forecast horizon to 2 years instead of the original 1 year
exp.predict_model(model, fh = np.arange(1, 25))

In [ ]:
# For models that do not produce a prediction interval --> returns NA values
model = exp.create_model("lr_cds_dt")
exp.predict_model(model, return_pred_int=True)

## Types of Window Splitters

### Sliding Window Splitter

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=fh, fold=fold, fold_strategy='sliding')
model = exp.create_model("ets")

### Expanding/Rolling Window

* They are identical

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=fh, fold=fold, fold_strategy='expanding')
model = exp.create_model("ets")

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=y, fh=fh, fold=fold, fold_strategy='rolling')
model = exp.create_model("ets")

## Error Handling

In [ ]:
try:
    exp = TimeSeriesExperiment()
    exp.setup(data=y, fh=17, fold=76, fold_strategy='expanding')
except ValueError as error:
    print(error)